<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/master/json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# fichier json
# dictionaire
import json
with open('/content/drive/My Drive/Colab Notebooks/PFE/twitter/updates.json') as file_json:
  data = json.load(file_json)

In [3]:
import pandas
dataset = []

# data['sites'].keys() : identifiants [1982,1981...] 
# data['sites'][identifiant][0] : fiabilite
# data['sites'][identifiant][1] : description
# data['sites'][identifiant][2] : nom
# data['sites'][identifiant][3] : nom miniscule et sans punctuation

for identifiant in data['sites'].keys():
  dataset.append([identifiant, 
                  data['sites'][identifiant][0], 
                  data['sites'][identifiant][1], 
                  data['sites'][identifiant][2], 
                  data['sites'][identifiant][3]])

In [4]:
len(dataset)

1343

In [ ]:
dataset

In [6]:
tableauSites = pandas.DataFrame(dataset, columns=['unique_id', 'fiabilité', 'description', 'name', 'name_low'])

In [7]:
tableauSites

,unique_id,fiabilité,description,name,name_low
0,1982,2,France-Soir était un journal fondé par Pierre ...,France Soir,france-soir
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",Régénère (Thierry Casasnovas),regenere-thierry-casasnovas
2,1980,2,Silvano Trotta est le créateur d’une société s...,Silvano Trotta,silvano-trotta
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa...",Jasper Mader,jasper-mader
4,1978,2,Une webtélé suisse créée notamment par Chloé F...,Agora TV,agora-tv
...,...,...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...,Dailymotion,dailymotion
1339,576,4,Une plateforme participative qui n'est pas ori...,Agoravox,agoravox
1340,575,4,9gag n'est pas une source d'information en soi...,9gag,9gag
1341,574,4,L'encyclopédie collaborative en ligne compile ...,Wikipédia,wikipedia


In [8]:
def fiabilite_to_label(fiabilite):
  fiabilites = {
     1 : "site parodique",
     2 : "non fiable",
     3 : "site à la fiabilité douteuse",
     4 : "fiable"
  }

  # dictionary.get(keyname, value)
  # keyname 	Required. The keyname of the item you want to return the value from
  # value 	Optional. A value to return if the specified key does not exist. Default value None
  return fiabilites.get(fiabilite, None)

labels = []
for identifiant in data['sites'].keys():
  label = fiabilite_to_label(data['sites'][identifiant][0])
  labels.append(label)


In [ ]:
labels

In [10]:
tableauSites['label'] = pandas.DataFrame(label for label in labels)

In [11]:
tableauSites

,unique_id,fiabilité,description,name,name_low,label
0,1982,2,France-Soir était un journal fondé par Pierre ...,France Soir,france-soir,non fiable
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",Régénère (Thierry Casasnovas),regenere-thierry-casasnovas,non fiable
2,1980,2,Silvano Trotta est le créateur d’une société s...,Silvano Trotta,silvano-trotta,non fiable
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa...",Jasper Mader,jasper-mader,non fiable
4,1978,2,Une webtélé suisse créée notamment par Chloé F...,Agora TV,agora-tv,non fiable
...,...,...,...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...,Dailymotion,dailymotion,fiable
1339,576,4,Une plateforme participative qui n'est pas ori...,Agoravox,agoravox,fiable
1340,575,4,9gag n'est pas une source d'information en soi...,9gag,9gag,fiable
1341,574,4,L'encyclopédie collaborative en ligne compile ...,Wikipédia,wikipedia,fiable


In [12]:
import re
dataset1 = []

# data['urls'].keys() : urls ['francesoir.fr', 'facebook.com/FranceSoir', 'twitter.com/france_soir', xxx]

for url in data['urls'].keys():
  if re.search('twitter.com/', url):
    dataset1.append([data['urls'][url],url])

# sort dataset1
dataset1.sort(reverse = True)

In [13]:
len(dataset1)

424

In [ ]:
dataset1

In [15]:
tableauSites['urls'] = pandas.DataFrame(url[1] for url in dataset1)

In [16]:
tableauSites

,unique_id,fiabilité,description,name,name_low,label,urls
0,1982,2,France-Soir était un journal fondé par Pierre ...,France Soir,france-soir,non fiable,twitter.com/france_soir
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",Régénère (Thierry Casasnovas),regenere-thierry-casasnovas,non fiable,twitter.com/ThierryRegenere
2,1980,2,Silvano Trotta est le créateur d’une société s...,Silvano Trotta,silvano-trotta,non fiable,twitter.com/silvano_trotta
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa...",Jasper Mader,jasper-mader,non fiable,twitter.com/emakrusi?lang=fr
4,1978,2,Une webtélé suisse créée notamment par Chloé F...,Agora TV,agora-tv,non fiable,twitter.com/llp_le_vrai
...,...,...,...,...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...,Dailymotion,dailymotion,fiable,NaN
1339,576,4,Une plateforme participative qui n'est pas ori...,Agoravox,agoravox,fiable,NaN
1340,575,4,9gag n'est pas une source d'information en soi...,9gag,9gag,fiable,NaN
1341,574,4,L'encyclopédie collaborative en ligne compile ...,Wikipédia,wikipedia,fiable,NaN


In [17]:
# ???
import re
import pandas as pd

dt = [] #Pour déterminer toutes les lignes dans le JSON qui ne sont pas des liens twitter
compteur = 0
for i in data["urls"].keys():
  if not(re.search('twitter', i)):
    dt.append(compteur)
  compteur = compteur+1

df = pd.DataFrame([data["urls"][i], i] for i in data["urls"].keys()) #On met tous les liens dans df
df = df.sort_index() #On trie car le JSON n'est pas trié
df = df.drop(index=dt) #On retire les lignes ne contenant pas de lien twitter, on utilise dt pour ça
df = df.reset_index() #On retrouve un idexage normal
df = df.drop(columns="index") #On retire la colonne qui s'est rajoutée à cause du reset_index()

#Liste de tous les identifiants qui n'ont pas de lien twitter
j = len(df.index)
dd = [] #Pour rajouter des lignes NaN aux domaines sans lien twitter

for i in range(j-1): #On créé un tableau avec tous les domaines sans lien twitter, tableau de forme [identifiant, "NaN"]
  for k in range(1, df[0][i] - df[0][i+1]): #Parfois plusieurs domaines à la suite n'ont pas de lien twitter, il faut rajouter une ligne pour chacun d'entre eux
    dd.append([df[0][i]-k, "NaN"]) # Cas où pour un identifiant particulié aucun lien twitter n'a été trouvé dans le fichier JSON

#Liste des tous les identifiants qui n'existent pas
dt = [] #Pour déterminer toutes les lignes dans le JSON font partis de "sites" mais qui n'ont pas de lien twitter
compteur = 0
j = 1982
for i in data["sites"].keys():
  for k in range(int(i)+1, j):
    dt.append([k, "NaN"])
  j = int(i)

for i in dt: #On retire de la liste des identifiants sans lien twitter les identifiants qui n'existent pas
  dd.remove(i)

df = df.append(pd.DataFrame(dd)) #On rajoute les nouvelles lignes
df = df.sort_values([0], ascending=False)
df = df.reset_index()
df = df.drop(columns="index")


#tableauSites['Twitter'] = df[1]
tableauSites['urls'] = df[1]
tableauSites

,unique_id,fiabilité,description,name,name_low,label,urls
0,1982,2,France-Soir était un journal fondé par Pierre ...,France Soir,france-soir,non fiable,twitter.com/france_soir
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",Régénère (Thierry Casasnovas),regenere-thierry-casasnovas,non fiable,twitter.com/ThierryRegenere
2,1980,2,Silvano Trotta est le créateur d’une société s...,Silvano Trotta,silvano-trotta,non fiable,twitter.com/silvano_trotta
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa...",Jasper Mader,jasper-mader,non fiable,NaN
4,1978,2,Une webtélé suisse créée notamment par Chloé F...,Agora TV,agora-tv,non fiable,NaN
...,...,...,...,...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...,Dailymotion,dailymotion,fiable,twitter.com/Fakir_
1339,576,4,Une plateforme participative qui n'est pas ori...,Agoravox,agoravox,fiable,NaN
1340,575,4,9gag n'est pas une source d'information en soi...,9gag,9gag,fiable,twitter.com/PaulJorion
1341,574,4,L'encyclopédie collaborative en ligne compile ...,Wikipédia,wikipedia,fiable,twitter.com/cahiersdufoot


In [20]:
!pip install xlwt

In [21]:
# DataFrame to Excel
tableauSites.to_excel('/content/drive/My Drive/Colab Notebooks/PFE/twitter/json.xlsx')